# Traitement des données avec pandas

- types **NumPy**
- renommer des colonnes : `df.rename(columns={...))`
- trouver/supprimer les données dupliquées : vdf.duplicated()` / `df.drop_duplicates()`
- trouver les NaN : `df.isna()` / `df.notna()` / `df.dropna()`
- travail sur les chaînes : `series.str.extract()`, `series.str.contains()`, `series.get_dummies()`
- mapping : `series.map()`
- changer le type d'une série (cast) : `df.astype(type)` / `pd.to_numeric()` / `pd.to_datetime()`
- remplacer n'importe quelle valeur : `df.replace({...})`
- remplacer les NaN : `df.fillna()`, `series.combine_first()`

In [ ]:
# imports
import numpy as np
import pandas as pd

#### Chargement et analyse des données

In [ ]:
pd.read_csv?

In [ ]:
# load data
df = pd.read_csv('people.csv')
df0 = df.copy()
df

In [ ]:
# info
df.info()

**Attention** : **pandas** utilise par défaut les types numériques les plus gourmands en mémoire.

In [ ]:
for subtype in ["int8", "int16", "int32", "int64"]:
    print(np.iinfo(subtype))

In [ ]:
for subtype in ["uint8", "uint16", "uint32", "uint64"]:
    print(np.iinfo(subtype))

In [ ]:
for subtype in ["float16", "float32", "float64"]:
    print(np.finfo(subtype))

In [ ]:
df = pd.read_csv('people.csv', dtype={"id":"uint16", "lon":"float16", "lat":"float16", "last_seen":"float32"})
df.info()

In [ ]:
df1 = pd.read_csv('people.csv', usecols=["id", "lon", "lat", "last_seen"])
df1.memory_usage(deep=True).sum()

In [ ]:
df1 = pd.read_csv('people.csv', usecols=["id", "lon", "lat", "last_seen"],
                  dtype={"id":"uint16", "lon":"float16", "lat":"float16", "last_seen":"float32"})
df1.memory_usage(deep=True).sum()

#### Renommage de la colonne 'email address'

In [ ]:
# renommer les colonnes
df = df.rename(columns={'email address': 'email'})
df.head()

#### Suppression des lignes dupliquées

- `duplicated()` : `True` ou `False` selon si une ligne est dupliquée
- `drop_duplicates()` : suppression des lignes dupliquées

In [ ]:
# lignes dupliquées
df.duplicated().value_counts()

In [ ]:
df.duplicated?

In [ ]:
# toutes les lignes dupliquées
df.loc[df.duplicated(keep=False)].sort_values('id')

In [ ]:
# suppression des lignes dupliquées
df = df.drop_duplicates()
len(df)

In [ ]:
# doc
df.drop_duplicates?

#### Analyse des données manquantes

`numpy.nan` est utilisé dans **pandas** pour représenter des valeurs manquantes.

In [ ]:
# Not A Number
np.nan

In [ ]:
# type
type(np.nan)

In [ ]:
# élément super absorbant
np.nan + 1

In [ ]:
# élément super absorbant
np.sqrt(np.nan)

In [ ]:
# élément super absorbant
np.nan == np.nan

In [ ]:
np.nan > 1

In [ ]:
# au passage, infinis numpy
np.NINF, np.inf

In [ ]:
np.inf > 1e100

In [ ]:
np.inf + 1e100

In [ ]:
np.inf == np.inf

In [ ]:
np.inf + np.NINF

In [ ]:
np.inf + np.inf > np.inf

#### Tests sur les données manquantes

- `isna()` ou `isnull()`
- `notna()` ou `notnull()`

In [ ]:
# ne fonctionne pas
df.loc[df['first_name']==np.nan]

In [ ]:
# chercher les first_name Nan
df.loc[df['first_name'].isna()]

In [ ]:
# sur tout le dataframe
df.isna()

In [ ]:
# chercher tous les lignes avec au moins un NaN
df.loc[df.isna().any(axis=1)]

In [ ]:
# suppression de toutes les lignes avec un NaN
df.dropna()

In [ ]:
# supprimer uniquement les lignes dont le first_name NaN 
df = df.dropna(subset=['first_name'])
len(df)

#### Ajout d'une colonne 'full_name'

In [ ]:
# 'full_name'  = 'first_name last_name'
df['full_name'] = df['first_name'] + ' ' + df['last_name']
df.head()

#### Analyse de la colonne 'address'

In [ ]:
# analyse de address
df['address'].value_counts()

#### Ajout des colonnes 'city' et 'country'

In [ ]:
# calcul de city et country à partir de address
df[['city', 'country']] = df['address'].str.extract('(.*), (.*)')
df.head()

#### Analyse des données

In [ ]:
# nunique : modalités par colonne
df.nunique()

#### Mapping du genre

In [ ]:
# analyse du gender
df['gender'].unique()

In [ ]:
# analyse du gender
df['gender'].value_counts()

In [ ]:
# traitement du gender
mapping = {'Female': 'F', 'Male': 'M', 'F': 'F', 'M': 'M'}
df['gender'] = df['gender'].map(mapping)
df.head()

In [ ]:
# dictionnaire incomplet
mapping0 = {'Female': 'F', 'Male': 'M'}
s = df0['gender'].map(mapping0)
df0['gender'].count(), s.count()

In [ ]:
s.value_counts(dropna=False)

In [ ]:
# traitement du gender, map() avec une Series
mapping = pd.Series({'Female': 'F', 'Male': 'M', 'F': 'F', 'M': 'M'})
mapping

In [ ]:
# traitement du gender avec une Series
df['gender'] = df['gender'].map(mapping)

In [ ]:
# au final
df['gender'].value_counts()

#### Analyse du genre

In [ ]:
# analyse gender NaN
len(df0.loc[df0['gender'].isna()])

In [ ]:
# analyse prénom avec gender NaN
df0.loc[df0['gender'].isna(), 'first_name'].nunique()

In [ ]:
# analyse gender
df0.loc[df0['gender'].isna(), 'first_name'].unique()

Compléter le genre :

1. Autocomplétion avec le fichier people.csv (mais très peu de cas)
2. Gender API : https://gender-api.com/fr (simple mais API payante si gros volumes + de 500/mois)
3. US SSA baby names : https://www.ssa.gov/oact/babynames/limits.html ("gratuit", stats à produire, éventuellement affiner par année de naissance)

#### Analyse de l'âge

`pandas.Series.astype()` : types

`pandas.to_numeric()` : data avec gestion des erreurs

In [ ]:
df.dtypes

In [ ]:
import re

re.search('[^0-9]', 'toto')

In [ ]:
# analyse de l'âge
df.loc[df['age'].astype(str).str.contains('[^0-9\.]'), 'age'].value_counts()

In [ ]:
pd.to_numeric?

In [ ]:
# traitement de l'âge
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df

In [ ]:
df["age"].unique()

In [ ]:
df.dtypes

#### Traitement des dates


`pandas.to_datetime()` : data, gestion des formats et des erreurs

`pandas.Series.combine_first()` : équivalent à `fillna()`

In [ ]:
cols_time = ['registration', 'last_seen']
df[cols_time]

In [ ]:
df[cols_time].dtypes

In [ ]:
pd.to_datetime?

In [ ]:
df0.loc[df0['last_seen'].isna()]

In [ ]:
# conversion des dates
df['registration'] = pd.to_datetime(df['registration'])
df['last_seen'] = pd.to_datetime(df['last_seen'], unit='s')
# si last_seen est NaN, prendre registration
df['last_seen'] = df['last_seen'].fillna(df['registration'])
# idem
df['last_seen'] = df['last_seen'].combine_first(df['registration'])

In [ ]:
df.head()

In [ ]:
df.info()

#### applymap() et apply() pour DataFrames

In [ ]:
# longueur de chaque élément passé en string
df.astype(str).applymap(len)

In [ ]:
# par défaut, la fonction s'applique aux colonnes => résultat = ligne
df.apply(len, axis=0)

In [ ]:
# exemple de ligne
df.iloc[0]

In [ ]:
# la fonction s'applique aux lignes => résultat = colonne
df.apply(len, axis=1)

In [ ]:
# exemple de colonne
df.iloc[:,0]

#### Analyse de currency

In [ ]:
# échantillon
np.random.seed(0)
df.sample(10)

#### Traitement de 'currency'

Produire une nouvelle colonne numérique 'money_eur'.

Pour la conversion USD/EUR, on utilise l'API https://api.exchangeratesapi.io/latest

In [ ]:
# API
import json

import requests

response = requests.get('https://open.er-api.com/v6/latest/EUR')
rates = json.loads(response.content)
rates

In [ ]:
df['money'].value_counts()

In [ ]:
df['currency'] = df['money'].str[0].map({'€': 'EUR', '$': 'USD'})
df.head()

In [ ]:
rates['rates']

In [ ]:
df0['money'].str[0].value_counts()

In [ ]:
# extraction de la currency
df['currency'] = df['money'].str[0].map({'€': 'EUR', '$': 'USD', '£': 'GBP'})
df['money_eur'] = df['money'].str[1:].str.replace(',', '.')  # extraction des derniers chars + , => .
df['money_eur'] = pd.to_numeric(df['money_eur'])  # conversion en nombre

# conversion des monnaies en euros
df['money_eur'] = df['money_eur'] / df['currency'].map(rates['rates'])
#np.random.seed(0)
#df.sample(10)
df.head()

In [ ]:
df['money_eur'] = df['money_eur'].round(2)
df.head()

#### Analyse des emails

On va utiliser des regex pour nettoyer les emails mais mieux vaut utiliser une librairie spécialisée. Par exemple, https://github.com/syrusakbary/validate_email

In [ ]:
# email NaN
df['email'].isna().sum()

In [ ]:
# suppression des emails absents
df = df.dropna(subset=['email'])
df

In [ ]:
# emails avec chars non admis
df.loc[df['email'].str.contains('[^A-Za-z0-9_\-%+.@]'), 'email'].values

In [ ]:
# suppression des blancs
df['email'] = df['email'].str.strip()
df.loc[df['email'].str.contains('[^A-Za-z0-9_\-%+.@]'), 'email']

In [ ]:
# regex pour vérifier les domaines
df.loc[~df['email'].str.contains('.+@.+\.[A-Za-z]{2,}$')]

In [ ]:
# emails avec noms de domaine invalides
df = df.loc[df['email'].str.contains('.+@.+\.[A-Za-z]{2,}$')]

In [ ]:
# emails avec aliases (char +)
df.loc[df['email'].str.contains('\+'), 'email']

In [ ]:
import re
re.sub(r'([^+]+)(?:\+.*)?(@.+)', r'\1\2', "a.gorz+alias@gmail.com")

In [ ]:
# suppression des aliases (char +)
s = df['email'].str.replace(r'([^+]+)(?:\+.*)?(@.+)', r'\1\2', regex=True)
s.loc[186]

In [ ]:
# [abc]
# [a-zA-Z]
# [^abc] : pas a, b ou c
# [^0-9] : pas digit
# '<[^>]+>' : tag HTML
# '<.+>'

In [ ]:
# caractères parenthèses
# \([0-9]+\)

In [ ]:
# parenthèses non capturante
# (?:regex)

In [ ]:
# back references
# (.*)@(.*)  \1  \2

In [ ]:
# suppresion des aliases (char +)
df['email'] = df['email'].str.replace(r'([^+]+)(?:\+.*)?(@.+)', r'\1\2', regex=True)

In [ ]:
# suppression des emails en double, on conserve la première ligne
df = df.drop_duplicates(subset=['email'])
df.sort_values('email')

#### Analyse de la colonne 'preference'

In [ ]:
# analyse de preference
df['preference'].nunique()

In [ ]:
# analyse de preference
df['preference'].value_counts()

In [ ]:
# modalités de preference
s = set()
df['preference'].apply(lambda x: s.update(x.split('/')))
s

In [ ]:
# ajout d'un booléen par preference
for x in sorted(s):
    df[x] = df['preference'].str.contains(x)
    
df

In [ ]:
# autre façon avec get_dummies
df['preference'].str.get_dummies(sep='/')

In [ ]:
# assignation des préférences
tab_preference = df['preference'].str.get_dummies(sep='/')
df[tab_preference.columns] = tab_preference.astype(bool)
df

In [ ]:
# OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
var = encoder.fit_transform(df[['preference']])
var.toarray()

In [ ]:
pd.DataFrame(var.toarray(), columns=encoder.get_feature_names())
# pd.DataFrame(var.toarray(), columns=encoder.get_feature_names_out())

#### clean

In [ ]:
import json

import requests
import pandas as pd

def clean_people(df):
    
    # suppression des lignes dupliquées
    df = df.drop_duplicates()
    
    # renommer les colonnes
    df = df.rename(columns={'email address': 'email'})
    
    # supprimer uniquement les lignes dont le first_name vaut NaN 
    df = df.dropna(subset=['first_name'])
    
    # ajout d'une colonne 'full_name'
    df['full_name'] = df['first_name'] + ' ' + df['last_name']

    # calcul de city et country à partir de address
    df[['city', 'country']] = df['address'].str.extract('(.*), (.*)')

    # traitement du gender
    mapping = {'Female': 'F', 'Male': 'M'}
    df['gender'] = df['gender'].map(mapping)
 
    # traitement de l'âge
    df['age'] = pd.to_numeric(df['age'], errors='coerce')

    # conversion des dates
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # si last_seen est NaN, prendre registration
    df['last_seen'] = df['last_seen'].fillna(df['registration'])

    # récupération des taux de change
    response = requests.get('https://open.er-api.com/v6/latest/EUR')
    rates = json.loads(response.content)

    # extraction de la currency
    df['currency'] = df['money'].str[0].map({'€': 'EUR', '$': 'USD'})
    df['money_eur'] = df['money'].str[1:].str.replace(',', '.')  # extraction des derniers chars + , => .
    df['money_eur'] = pd.to_numeric(df['money_eur'])  # conversion en nombre

    # conversion des monnaies en euros
    rates['rates']['EUR'] = 1.0  # ajour de EUR pour pouvoir utiliser map()
    df['money_eur'] = df['money_eur'] * df['currency'].map(rates['rates'])

    # suppression des emails absents
    df = df.dropna(subset=['email'])

    # suppression des blancs
    df['email'] = df['email'].str.strip()

    # emails avec noms de domaine valides
    df = df.loc[df['email'].str.contains('.+@[A-Za-z0-9_\-.]+\.[A-Za-z]{2,}')]
    
    # extraction des aliases (char +)
    df['email'] = df['email'].str.replace(r'([^+]+)(?:\+.*)?(@.+)', r'\1\2', regex=True)

    # suppression des emails en double, on conserve la première ligne
    df = df.drop_duplicates(subset=['email'])

    # assignation des préférences
    tab_preference = df['preference'].str.get_dummies(sep='/')
    df[tab_preference.columns] = tab_preference.astype(bool)

    return df

In [ ]:
# run
import pandas as pd

df0 = pd.read_csv('people.csv')
print(df0.shape)

df = clean_people(df0)
print(df.shape)

In [ ]:
# method chaining

def clean_people2(df):
    
    # récupération des taux de change
    response = requests.get('https://open.er-api.com/v6/latest/EUR')
    rates = json.loads(response.content)
    rates['rates']['EUR'] = 1.0  # ajour de EUR pour pouvoir utiliser map()
    
    df = (df
          .drop_duplicates()
          .rename(columns={'email address': 'email'})
          .dropna(subset=['first_name'])
          .assign(full_name=lambda df_: df_.first_name + ' ' + df_.last_name,
                  gender=lambda df_: df_.gender.map({'Female': 'F', 'Male': 'M'}),
                  age=lambda df_: pd.to_numeric(df_.age, errors='coerce'),
                  registration=lambda df_: pd.to_datetime(df.registration),
                  last_seen=lambda df_: pd.to_datetime(df.last_seen, unit='s'))
          .assign(last_seen=lambda df_: df.last_seen.fillna(df_.registration))
          .pipe(lambda df_: df_.assign(**df_.address.str.extract('(?P<city>.*), (?P<country>.*)')))
          .assign(currency=lambda df_: df_.money.str[0].map({'€': 'EUR', '$': 'USD'}),
                  money_eur=lambda df_: df_.money.str[1:].str.replace(',', '.'))
          .assign(money_eur=lambda df_: pd.to_numeric(df_.money_eur) * df_.currency.map(rates['rates']))
          .dropna(subset=['email'])
          .assign(email=lambda df_: df_.email.str.strip())
          .assign(email=lambda df_: df_.email.str.replace(r'([^+]+)(?:\+.*)?(@.+)', r'\1\2', regex=True))
          .loc[lambda df_: df_.email.str.contains('.+@[A-Za-z0-9_\-.]+\.[A-Za-z]{2,}')]
          .drop_duplicates(subset=['email'])
          .pipe(lambda df_: df_.assign(**df_.preference.str.get_dummies(sep='/').astype(bool)))
         )

    return df

In [ ]:
# run
import pandas as pd

df0 = pd.read_csv('people.csv')
print(df0.shape)

df = clean_people2(df0)
print(df.shape)

#### Visualisation des tables

Voir : https://pandas.pydata.org/docs/user_guide/style.html

Taux de remplissage d'un dataframe en bar chart

In [ ]:
df = pd.read_csv('people.csv')

(df.notna()
 .sum()
 .to_frame()
 .rename({0: "completion"}, axis=1)
 .style
 .bar(color='lightgreen')
)

Taux de remplissage d'un dataframe en color map

In [ ]:
(df.notna()
 .sum()
 .mul(100/len(df))
 .to_frame()
 .rename({0: "completion"}, axis=1)
 .style.background_gradient(cmap="RdYlGn")
 .format("{:.1f}%")
)

**Exercice**

1. Téléchargez le fichier Excel "FranceTHD_Open_Data_Observatoire_2017_T2.xlsx" sur le niveau des débits sur les réseaux d'accès à Internet : ADSL, câble, Fibre FttH (T2 2015 - T2 2017) de la page : https://www.data.gouv.fr/fr/datasets/niveau-des-debits-sur-les-reseaux-dacces-a-internet-adsl-cable-fibre-ftth-t2-2015-t2-2017/

2. Chargez avec pd.read_excel() dans un DataFrame le dernier onglet "2017 T2" en mesurant le temps avec %%time en première instruction de cellule.

3. Modifiez le nom des 4 premières colonnes en : 'code INSEE', 'commune', 'département', 'nb locaux' par exemple.

4. Sauvegardez le DataFrame avec pd.to_pickle().

5. Rechargez le DataFrame à partir du fichier pickle en mesurant le temps avec %%time en première instruction de cellule et comparez.

6. Effectuez une opération de sélection sur les communes : par exemple, les communes qui commencent par "SAINT".

7. Diagnostiquez le message d'erreur.

8. Corrigez le DataFrame.

In [ ]:
%%time
df = pd.read_excel('FranceTHD_Open_Data_Observatoire_2017_T2.xlsx',
               sheet_name=-1,
               header=1)
df

In [ ]:
df = df.rename(columns={'Unnamed: 0':'Code INSEE','Unnamed: 1':'Commune','Unnamed: 2':'Département','Unnamed: 3':'nb locaux'})
df.head()

In [ ]:
df.to_pickle('THD.pkl')

In [ ]:
%%time
df = pd.read_pickle('THD.pkl')
df.shape

In [ ]:
df.info()

In [ ]:
df.loc[df['Commune'].str.startswith('Saint')]

In [ ]:
df.loc[df['Commune'].str.startswith('Saint', na=False)]

In [ ]:
df['Commune'].apply(type).value_counts()

In [ ]:
df.loc[df['Commune'].apply(lambda x: isinstance(x, bool))]

In [ ]:
df.loc[df['Commune']==False, 'Commune'] = 'Faux'

In [ ]:
df.loc[df['Commune'].str.startswith('Saint')]

#### Etudier les multiples options de read_csv()

En particulier:

<pre>
pd.read_csv(
    <strong>filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]],</strong>
    <strong>sep=',',</strong>
    delimiter=None,
    <strong>header='infer',</strong>
    <strong>names=None,</strong>
    <strong>index_col=None,</strong>
    <strong>usecols=None,</strong>
    squeeze=False,
    prefix=None,
    mangle_dupe_cols=True,
    <strong>dtype=None,</strong>
    <strong>engine=None,</strong>
    <strong>converters=None,</strong>
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    <strong>skiprows=None,</strong>
    <strong>skipfooter=0,</strong>
    <strong>nrows=None,</strong>
    <strong>na_values=None,</strong>
    <strong>keep_default_na=True,</strong>
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    <strong>parse_dates=False,</strong>
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    cache_dates=True,
    iterator=False,
    <strong>chunksize=None,</strong>
    compression='infer',
    <strong>thousands=None,</strong>
    <strong>decimal='.',</strong>
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
    escapechar=None,
    comment=None,
    encoding=None,
    encoding_errors=None,
    dialect=None,
    error_bad_lines=True,
    warn_bad_lines=True,
    on_bad_lines=None,
    delim_whitespace=False,
    low_memory=True,
    memory_map=False,
    float_precision=None,
    storage_options=None
)
</pre>

#### Analyse automatique avec pandas_profiling

https://github.com/pandas-profiling/pandas-profiling

**ATTENTION, il vaut mieux installer `pandas_profiling` dans un nouvel environnement**

<pre>
conda create --name profiling

activate profiling OU conda activate profiling

conda install -c conda-forge pandas-profiling
</pre>

In [ ]:
# profiling raw people
from pandas_profiling import ProfileReport

df = pd.read_csv('people.csv')

profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)

profile.to_file("people.html")

In [ ]:
# profiling clean people
profile = ProfileReport(clean_people(df), title='Pandas Profiling Report', explorative=True)

profile.to_file("clean_people.html")